In [ ]:
import os
import glob
import multiprocessing
from UUID import conversionUUID
from Splitting import split
import time
from Conversion import convert_tiff_folder_to_pdf
from pdf_processor import pdf_segregate
from FileMerger import pdf_merger
from BOL_EXTRACTION import bol_main
from INVOICE_EXTRACTION import inv_main
def process_pdf(file_path):
    try:
        print(f"Processing {file_path}")
        pdf_segregate(file_path)
    except Exception as e:
        return (file_path, e)

temp_folder = os.getcwd() + "/Base_Folder"
convert_tiff_folder_to_pdf(temp_folder, temp_folder)

file_pattern = "*.pdf"
bol_folder_path = os.getcwd() + "/Final/Bill of Lading/"
bol_file_list = glob.glob(bol_folder_path + '/' + file_pattern)
drayinv_folder_path = os.getcwd() + "/Final/DrayInv/"
inv_file_list = glob.glob(drayinv_folder_path + '/' + file_pattern)

def pdf_main():
    conversionUUID()
    split()
    file_pattern = "*.pdf"
    folder_path = os.getcwd() + "/Split Folder/"
    file_list = glob.glob(folder_path + '/' + file_pattern)

    num_processes = 7
    with multiprocessing.Pool(processes=num_processes) as pool:
        results = pool.map(process_pdf, file_list)

    for result in results:
        if isinstance(result, tuple) and len(result) == 2:
            file_path, exception = result
            print(f"Exception occurred while processing {file_path}: {exception}")

    directories = {"Classification/bill of lading": "Final/Bill of Lading", "Classification/Corresp": "Final/Corresp", "Classification/DrayInv": "Final/DrayInv", "Classification/cash receipt": "Final/CashReceipt", "Classification/Lumper": "Final/Lumper", "Classification/Weight Ticket": "Final/WeightTicket"}
    pdf_merger(directories)
    pool.close()
    pool.join()

p1 = multiprocessing.Process(target=bol_main)
p2 = multiprocessing.Process(target=inv_main)

if __name__ == "__main__":
    start_time = time.time()
    pdf_main()
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Total execution time: {execution_time} seconds")

